# Importar librerias

In [1]:
c = 7 # Valor para variables climaticas

In [2]:
import time
start = time.time()
#importar datos y librerias

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime
from scipy import signal
from scipy import special
from scipy.ndimage.interpolation import shift
from statsmodels.tsa.seasonal import seasonal_decompose

from pmdarima.arima import auto_arima
from statsmodels.tsa.arima_model import ARIMA
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
%matplotlib inline

In [3]:
# Plotly
import chart_studio.plotly as ply
import cufflinks as cf
cf.go_offline()

# Funciones

In [4]:
# para calcular el mape
def Mape(y,y_pred):
    return np.mean(np.abs((y-y_pred)/y))*100

# para hacer kronecker por fila
def kronecker(data1:'Dataframe 1',data2:'Dataframe 2'):
    Combinacion = pd.DataFrame(); d1 = pd.DataFrame()
    for i in data2.columns:
        d1 = data1.multiply(data2[f"{i}"] , axis="index")
        d1.columns = [f'{i}_{j}' for j in data1.columns]
        Combinacion = pd.concat([Combinacion, d1], axis = 1)
    return Combinacion

# para crear variables difusas
def triangular():
    # Se crea x
    ma = pd.DataFrame();
    x = consumofeb.index.dayofyear.tolist()
    # Se crean variables a, b y c
    height = difs()
    for hei in height:
        ba = pd.DataFrame(x, columns=["x"])
        ba["a"] = height[hei][0]
        ba["b"] = height[hei][1]
        ba["c"] = height[hei][2]
        # Variable difusa
        ca = pd.DataFrame()
        ca["bo"] = (ba.x-ba.a)/(ba.b-ba.a)
        ca["ba"] = (ba.c-ba.x)/(ba.c-ba.b)
        ca = ca.min(axis = 1)
        ca[ca<0]=0
        ma = pd.concat([ma, ca], axis = 1) # Se concatenan
        
    ma.index = consumofeb.index
    ma.columns = height.keys()
    return ma

In [5]:
# triangular()

# Cargar datos

In [38]:
def cargar():
    # variables climaticas
    consumofeb=pd.read_excel(r'Consumo_feb.xlsx').set_index("fecha").loc["2007-01-01":"2020-03-30"] # desde el 2007
    consumofeb=consumofeb.drop(["prom","Lluvia","Velocidad_viento"],1) # variables no importantes para el modelo
    #variables de consumo MWh
    consumo=pd.read_excel(r'Data1.xlsx').set_index("fecha").loc["2007-01-01":"2020-03-30"]  # consumo de energia en MWh
    while True:
        if len(consumofeb.index) != len(consumo.index): 
            raise RuntimeError("Climatic variable and consumption variable length does not match")
            break
        else: return consumo, consumofeb

consumo, consumofeb = cargar()

In [39]:
if len(consumofeb.index) == len(consumo.index): print("ok")

ok


 # Variables difusas

In [40]:
def difs():
    # Inicializar posicion de datos variables difusas

    inv1 = [310.5-365.25, 8.6529, 138.941176470588]
    pr = [8.6529, 138.941176470588, 221.9375]
    vr = [138.941176470588, 221.9375, 310.5]
    ot = [221.9375, 310.5, 365.25+8.6529]
    ot2 = [221.9375-365.25, 310.5-365.25, 8.6529]
    inv2 = [310.5, 365.25+8.6529, 365.25+138.941176470588]

    height = {'Invierno1': inv1,'Primavera': pr,'verano': vr,'Otoño': ot,'Otoño2': ot2,'Invierno2': inv2,} 
    return height 

-  Para graficar

In [41]:
# triangular().iplot()

# Variables climaticas

In [42]:
def pba(j, n):
    # armonicos
    f, asd = signal.periodogram(consumofeb[j], 1)
#     plt.figure()
#     plt.plot(1/f, asd)
    picos = pd.DataFrame(asd, 1/(f), columns=["potencia"])
    picos = picos.sort_values(by="potencia",ascending=False).reset_index().head(12)
    picos.columns = ["periodo", "potencia"]
    #fourier
    sencos = pd.DataFrame(index = consumofeb.index)
    t = np.arange(1,len(consumofeb)+1)
    sencos["t"]=t
    for i  in  picos.periodo:
            sencos[f"{i:.2f}_sen"] = np.abs(np.sin(((2*np.pi)/(i))*t))
            sencos[f"{i:.2f}_cos"] = np.abs(np.cos(((2*np.pi)/(i))*t))
    sencos['ones']=1
    sencos['sen1']=np.abs(np.sin(((2*np.pi)/(365.25))*t))
    sencos['cos1']=np.abs(np.cos(((2*np.pi)/(365.25))*t))
    
    sencos=kronecker(sencos, triangular())
    #regresion
    X_train, X_test, y_train, y_test = train_test_split(sencos, consumofeb[j], test_size=n, shuffle = False)
    
    reg = LinearRegression(n_jobs=-1).fit(X_train, y_train)
    y_predict=reg.predict(X_test)
    
#     plt.figure()
#     plt.plot(y_test , label="real")
#     plt.plot(y_test.index, y_predict,label="predicción")

#     plt.figure()
#     plt.plot(y_train)
#     plt.plot(y_train.index, reg.predict(X_train))
#     print(Mape(y_train,reg.predict(X_train),1))
    
    nov=pd.DataFrame(np.concatenate([reg.predict(X_train),y_predict]),index=consumofeb.index,columns=[f"{j}"])

    return nov

In [43]:
# pba("max", 33)

In [44]:
def modelosclima(n):
    ca = pd.DataFrame(index = consumo.index)
    for i in ["mín", "max", "Nubosidad"]:
        ca = pd.concat([ca, pba(i, n)], axis = 1)
    return ca

In [45]:
# Se corre y guarda en una variable los datos de clima predichos
modclima = modelosclima(c)

# Varibles festivos

In [46]:
def festivos():
    festivos=pd.read_excel(r'Festivos.xlsx')
    festivos2=pd.read_excel(r'Festivos2.xlsx')

    Dum = pd.DataFrame(index = consumo.index)

    for col in festivos.columns:
        Dum[f"{col}"] = consumo.index.isin(festivos[f"{col}"])
        Dum[col] = Dum[col].replace([False, True],[0, 1])
        clean = Dum[col].values
        Dum[col] = Dum[col]+shift(clean, 1, cval=0)*.4
        Dum[col] = Dum[col]+shift(clean, -1, cval=0)*.4
        Dum[col] = Dum[col]+shift(clean, 2, cval=0)*.1
        Dum[col] = Dum[col]+shift(clean, -2, cval=0)*.1
        Dum[col] = Dum[col]/2

    Dum["ones"] = 1
    Dum["t"]=np.arange(1,len(consumo.index)+1)    
    
    for col in festivos2.columns:
        Dum[col] = consumo.index.isin(festivos2[col])
    Dum = Dum.replace([False, True],[0, 1])
    
    return Dum

# Todas las variables dependientes

In [47]:
def dummies(n):   
    # variables eureka y clima
    consumo2 = pd.DataFrame(index=consumo.index)
    consumo2["num"] = np.arange(1,len(consumo.index)+1)
    consumo2["day"] = consumo.index.weekday
    consumo2["eureka1"] = consumo2.day**3
    consumo2["eureka2"] = modclima["max"]*consumo2.num
    consumo2["month"] = consumo.index.month
    X = consumo2.join(modclima)
    # normalizacion eureka-clima
    X_norm = X/X.max()
    
    # variables gplearn
    X_norm["gplearn1"]=np.cos(X_norm.day)*X_norm.num
    X_norm["gplearn2"]=X_norm.num*X_norm["mín"]
    X_norm["gplearn3"]=3.319**X_norm["max"]*np.cos(X_norm.day)
    X_norm["gplearn8"]=3.319**X_norm["max"]*X_norm.num 
    X_norm["gplearn4"]=np.cos(X_norm.day)*np.sin(X_norm.day)
    X_norm["gplearn6"]=X_norm["max"]**2*X_norm["mín"]**2
    X_norm["gplearn7"]=1/np.cos(X_norm.day)
    X_norm["gplearn10"]=X_norm["max"]*X_norm["mín"]**3*np.cos(X_norm.month)**2
    # variables dummies
    X_norm = pd.get_dummies(X_norm, columns=["month", "day"], prefix=["month", "day"], drop_first=True)
    
    X_norm=X_norm.drop(["mín", "max", "Nubosidad"],axis=1) 

    # variables festivos y difusa de clima
    X_norm = X_norm.join(festivos())
    X_norm = X_norm.join(triangular())
    
    # variables entre y fin
    X_norm["fin"]=consumo.index.weekday
    X_norm["entre"]=consumo.index.weekday
    X_norm["fin"]=X_norm["fin"].replace([0,1,2,3,4,5,6],[.5,0,0,0,.5,1,1])
    X_norm["entre"]=X_norm["entre"].replace([0,1,2,3,4,5,6],[.5,1,1,1,.5,0,0])
    return X_norm

In [48]:
beg = time.time()
dummies(7)
end = time.time()
end - beg

0.4523797035217285

# Para separar $x$ y $y$

In [49]:
def separar(n, X, y):
    # separar X y Y
    X_train, X_test, y_train, y_test = train_test_split(X, y, \
                                                  test_size = n, shuffle = False)
    return (X_train, X_test), (y_train, y_test)

# Primer regresion

In [50]:
def reg(n, pre_forc): # n = 7 , pre_for = "general" o "forecasting"
    # cargar X y Y
    y = consumo; X = dummies(n)
    # Se obtienen los datos separados
    Xo, yo = separar(n, X, y)  
    # Decision "general" "forcasting"
    while True:
        if pre_forc == "general": 
            pre_forc = 0
            break
        elif pre_forc == "forecasting": 
            pre_forc = 1
            break
        else: 
            raise NameError(f"expected 'general' or 'forecasting', got '{pre_forc}' instead")
            break
            
    # regresion y predeccion    
    reg = LinearRegression(n_jobs =-1).fit(Xo[0], yo[0])
    y_predict=reg.predict(Xo[pre_forc])
    
    # dataframe real-predict- resta
    comp = pd.DataFrame(y_predict, columns = ["predict"],index = yo[pre_forc].index)
    comp["real"] = yo[pre_forc]; comp["resta"] = comp.real - comp.predict
    return comp
    

- Para graficar

In [51]:
def visual1(n):
    regio = reg(n, "general")
    rego = regio[["predict", "real"]]
    
    rego.iplot(title = f"general: {Mape(*rego.values.T)}")
    regio.resta.iplot(title = "resta")
    
    regal = reg(n, "forecasting")[["predict", "real"]]
    regal.iplot(title = f"forecasting: {Mape(*regal.values.T)}")

In [52]:
# visual1(7)

# Periodos en residuales

In [53]:
def perio(n, fg):
    # Periodograma de resta real - pred
    reg1 = reg(n, fg)
    
    f, asd = signal.periodogram(reg1.resta, 1)
    
#     # graficar
#     plt.plot(1/f, asd); plt.xlabel('periodo'); plt.ylabel('PSD')

    # DataFrame con columna de potencia y columna de 
    picos = pd.DataFrame(asd, columns=["potencia"])
    picos["periodo"] = 1/f
    picos = picos.sort_values(by="potencia",ascending=False).reset_index(drop = True).head(12)
    return picos[2:12]

In [54]:
# perio(7, "general")

# Kronecker Sencos-dummies

In [55]:
def kron(n):
    # Se llama a X y a Y
    X = dummies(n) 
    # Se crea dataframe de fourier
    sencos = pd.DataFrame()
    t = np.arange(1,len(consumo.MWh)+1)
    sencos["t"]=t
    
    # Mando llamar la funcion de armonicos
    p = perio(n, "general")
    
    # fourier
    for i  in  p.periodo:
            sencos["{}_sen".format(i)] = np.abs(np.sin(((2*np.pi)/(i))*t))
            sencos["{}_cos".format(i)] = np.abs(np.cos(((2*np.pi)/(i))*t))
    sencos['ones']=1
    sencos['sen']=np.abs(np.sin(((2*np.pi)/14)*t))
    sencos['cos']=np.abs(np.cos(((2*np.pi)/14)*t))
    sencos['sen1']=np.abs(np.sin(((2*np.pi)/(365.25))*t))
    sencos['cos1']=np.abs(np.cos(((2*np.pi)/(365.25))*t))
    sencos['sen2']=np.abs(np.sin(((2*np.pi)/(28))*t))
    sencos['cos2']=np.abs(np.cos(((2*np.pi)/(28))*t))
    
    sencos.index =consumo.index
    
    # Kronecker
    combination=kronecker(X, sencos)
    return combination.join(modclima)

# Segunda regresion

In [56]:
def reg2(n, pre_forc): # n = 7 , pre_for = "general" o "forcasting"
    # Se llama a X y a Y
    X = kron(n); y = consumo
    
    # split
    Xo, yo = separar(n, X, y) 
    
    # Decision "general" "forcasting"
    while True:
        if pre_forc == "general": 
            pre_forc = 0
            break
        elif pre_forc == "forecasting": 
            pre_forc = 1
            break
        else: 
            raise NameError(f"expected 'general' or 'forecasting', got '{pre_forc}' instead")
            break
        
    # regresion y predeccion    
    reg = LinearRegression(n_jobs =-1).fit(Xo[0], yo[0])
    y_predict=reg.predict(Xo[pre_forc])
    
    # dataframe real-predict- resta
    comp = pd.DataFrame(y_predict, columns = ["predict"],index = yo[pre_forc].index)
    comp["real"] = yo[pre_forc]; comp["resta"] = comp.real - comp.predict
    return comp

- para graficar

In [57]:
def visual2(n):
    regio = reg2(n, "general")
    rego = regio[["predict", "real"]]
    
    rego.iplot(title = f"general: {Mape(*rego.values.T)}")
    regio.resta.iplot(title = "resta")
    
    regal = reg2(n, "forecasting")[["predict", "real"]]
    regal.iplot(title = f"forecasting: {Mape(*regal.values.T)}")

In [58]:
# visual2(10)

# Arima

In [59]:
def aroma(n):
    # llamar X
    X = festivos().join(triangular())[:-n]
    # llamar y
    result = reg2(n, "general").resta
    # arima
    step_wise=auto_arima(result,exog=X,start_p=1, start_q=1, max_p=7, max_q=7, max_d=1,
                     trace=True, error_action='ignore', suppress_warnings=True, stepwise=True,m=1,
                    seasonal=False,Stationary=True)
    
    mod = ARIMA(result, order=step_wise.order)
    results = mod.fit()

    return results # modelo

In [60]:
# aroma(7)

# Tercer regresion

In [61]:
def reg3(n):
    X = kron(n)
    X_train, X_test, y_train, y_test = train_test_split(X, consumo, test_size = n, shuffle = False)
    #arima
    results = aroma(n)   
    # prediccion con arima
    extra=results.predict(len(y_train),len(y_train)+n)[1:]
    
    # regresion con lasso
    lasso_reg = Lasso(normalize=True,max_iter=1000).fit(X_train,y_train)
    y_pred_lass = lasso_reg.predict(X_test)
    print(len(lasso_reg.coef_))

    # predict, real, resta
    comp = pd.DataFrame(y_pred_lass + extra.values, columns = ["predict"],index = y_test.index)
    comp["real"] = y_test; comp["resta"] = comp.real - comp.predict
    return comp

In [62]:
beg = time.time()
ba = reg3(7)
end = time.time()
print(f" timepo de corrida: {(end - beg)/60} min")

Performing stepwise search to minimize aic
Fit ARIMA: (1, 0, 1)x(0, 0, 0, 0) (constant=True); AIC=86488.983, BIC=86514.914, Time=0.759 seconds
Fit ARIMA: (0, 0, 0)x(0, 0, 0, 0) (constant=True); AIC=88012.738, BIC=88025.704, Time=0.135 seconds
Fit ARIMA: (1, 0, 0)x(0, 0, 0, 0) (constant=True); AIC=86499.099, BIC=86518.548, Time=0.298 seconds
Fit ARIMA: (0, 0, 1)x(0, 0, 0, 0) (constant=True); AIC=86754.370, BIC=86773.818, Time=0.513 seconds
Fit ARIMA: (0, 0, 0)x(0, 0, 0, 0) (constant=False); AIC=88010.738, BIC=88017.221, Time=0.064 seconds
Fit ARIMA: (2, 0, 1)x(0, 0, 0, 0) (constant=True); AIC=86126.569, BIC=86158.983, Time=6.418 seconds
Fit ARIMA: (2, 0, 0)x(0, 0, 0, 0) (constant=True); AIC=86485.311, BIC=86511.242, Time=0.447 seconds
Fit ARIMA: (3, 0, 1)x(0, 0, 0, 0) (constant=True); AIC=86124.528, BIC=86163.425, Time=4.068 seconds
Fit ARIMA: (3, 0, 0)x(0, 0, 0, 0) (constant=True); AIC=86451.758, BIC=86484.172, Time=0.568 seconds
Fit ARIMA: (4, 0, 1)x(0, 0, 0, 0) (constant=True); AIC=8

C:\Users\alber\AppData\Roaming\Python\Python37\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning:

No frequency information was provided, so inferred frequency D will be used.



1711
 timepo de corrida: 2.4736316164334613 min


C:\Users\alber\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 1176063757.0596924, tolerance: 218537599.8996125



- para graficar

In [63]:
ai = Mape(*ba[["predict", "real"]].values.T)
ba[["predict", "real"]].iplot(title = f"Mape: {ai:.3f}")

In [64]:
ba

,predict,real,resta
fecha,,,
2020-03-24,206166.974842,206724.150000,557.175158
2020-03-25,207860.559479,207132.601673,-727.957806
2020-03-26,207713.293786,205825.952954,-1887.340831
2020-03-27,206305.327905,202961.538433,-3343.789471
2020-03-28,196211.149937,192610.269605,-3600.880333
2020-03-29,174346.935608,168345.521000,-6001.414608
2020-03-30,199243.755472,191108.613000,-8135.142472
